In [ ]:

NUM_PROC_MAP = 150 # expect 30 min with single process
PUSH_TO_HF = False

DEBUG = True
EPOCHS = 10
HF_USERNAME = None

import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

import numpy as np

from scipy.stats import norm
from scipy.stats import f
from scipy.stats import lognorm
import plotting
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
import torch
import datasets
from datasets import load_dataset




DEBUG = True

import torch
from scipy.stats import kendalltau
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


from baseline_curricula import validate_training_duration_limitation
from itertools import product, chain
import config 
from util import get_curriculum

jobs =[(dataset, curriculum,model_type) for dataset, model_type, curriculum in product(config.datasets, config.model_types, config.baseline_curricula)]
jobs.extend([(dataset, model_type + curriculum,model_type) for dataset, model_type, curriculum  in (product(config.datasets, config.model_types, config.influence_curricula))])


In [ ]:
from util import get_curriculum


In [ ]:

from itertools import product
import config 

In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
word_count = lambda d: {"words": len(d["text"].split())}
cache_path = (os.path.join("./plotting_notebooks/cache/","curricula_metrics_counts.pkl"))


def get_tokens_seen(word_counts_dataset, curriculum):
    indices = torch.cat(curriculum).flatten().tolist()
    selected = word_counts_dataset.select(indices)
    return selected["words"]

if not os.path.exists(cache_path):
    dataset_cache = {}
    unique_datasets = set(dataset for dataset, _,_ in jobs)

    # Preprocess and cache each dataset once
    for dataset_name in tqdm(unique_datasets, desc="Preprocessing datasets"):
        dataset = load_dataset(dataset_name)["train"]
        word_counts_dataset = dataset.map(word_count, num_proc=100)
        dataset_cache[dataset_name] = {
            "word_counts": word_counts_dataset,

        }

    def process_experiment(args):
        dataset_name, curriculum_name, model_type,  data_info = args
        curriculum = get_curriculum(dataset_name, curriculum_name)
        
        r = {
            "dataset": dataset_name,
            "model_type": model_type,
            "curriculum": curriculum_name,
            "tokens seen by model": get_tokens_seen(data_info["word_counts"], curriculum),
        }
        # print( r, flush=True)
        return r

    job_args = [(dataset, curriculum, model_type, dataset_cache[dataset]) for dataset, curriculum, model_type in jobs]

    with Pool(10) as pool:
        results = list(tqdm(pool.imap(process_experiment, job_args), total=len(job_args)))

    df = pd.DataFrame(results)
    df.to_pickle(cache_path)

else:
    print("Reusing cached results")

df = pd.read_pickle(cache_path)


In [ ]:
df["tokens seen by model"] = df["tokens seen by model"].apply(lambda x: [np.mean(a) for a in np.array_split(x,1000)])

In [ ]:
import util
df["curriculum"] = df["curriculum"].apply(util.rename)
df["dataset"] = df["dataset"].apply(util.rename_dataset)

In [ ]:
df

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_tokens_over_time(df, save_path="./autogenerated_figures/tokens_over_time_facet_grid.pdf"):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    df_exploded = df[ (df["dataset"] != "$D_{equitoken}$")].explode('tokens seen by model').reset_index(drop=True)

    g = sns.FacetGrid(df_exploded, row="dataset", col="curriculum", height=5, aspect=1.5, sharey=True, margin_titles=True)

    def plot_func(data, **kwargs):
        x = data.groupby('model_type').cumcount()
        sns.lineplot(x=x, y=data['tokens seen by model'], hue=data['model_type'], markers=False, palette="Set2", legend="full", linewidth=0.7, **kwargs)

    g.map_dataframe(plot_func)

    g.set_titles(row_template="{row_name}", col_template="{col_name}")

    for ax in g.axes.flat:
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xticks([])

    g.fig.text(0.5, -0.10, 'Time $\\rightarrow$', ha='center', fontsize=12, fontweight='bold')
    g.fig.text(-0.02, 0.5, 'Tokens per doc', va='center', rotation='vertical', fontsize=12, fontweight='bold')

    g.fig.subplots_adjust(wspace=0)
    g.fig.subplots_adjust(hspace=0)

    g.fig.set_size_inches(10, 2.5)
    g.tight_layout(pad=0)

    plt.savefig(save_path, dpi=600, bbox_inches='tight')
    plt.show()

plot_tokens_over_time(df)


In [ ]:
df

In [ ]:


df["tokens seen by model var"] = df['tokens seen by model'].apply(np.var)
pd.DataFrame(df.groupby(["curriculum"])['tokens seen by model var'].mean().sort_values())

In [ ]:
df

In [ ]:
benchmark_results = pd.read_pickle("./plotting_notebooks/cache/benchmark_results.pkl")
benchmark_results

In [ ]:
df

In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
word_count = lambda d: {"words": len(d["text"].split())}
cache_path = (os.path.join("./plotting_notebooks/cache/","curricula_metrics.csv"))


def count_tokens_seen(word_counts_dataset, curriculum):
    indices = torch.cat(curriculum).flatten().tolist()
    selected = word_counts_dataset.select(indices)
    return sum(selected["words"])

if not os.path.exists(cache_path):
    dataset_cache = {}
    unique_datasets = set(dataset for dataset, _,_ in jobs)

    # Preprocess and cache each dataset once
    for dataset_name in tqdm(unique_datasets, desc="Preprocessing datasets"):
        dataset = load_dataset(dataset_name)["train"]
        word_counts_dataset = dataset.map(word_count, num_proc=100)
        dataset_cache[dataset_name] = {
            "word_counts": word_counts_dataset,
            "tokens_in_dataset": sum(word_counts_dataset["words"]),
            "dataset_len": len(dataset),
        }

    def process_experiment(args):
        dataset_name, curriculum_name, model_type,  data_info = args
        curriculum = get_curriculum(dataset_name, curriculum_name)
        
        r = {
            "dataset": dataset_name,
            "model_type": model_type,
            "curriculum": curriculum_name,
            "documents in dataset": data_info["dataset_len"],
            "documents seen by model": sum(len(epoch) for epoch in curriculum),
            "tokens in dataset": data_info["tokens_in_dataset"],
            "tokens seen by model": count_tokens_seen(data_info["word_counts"], curriculum),
        }
        # print( r, flush=True)
        return r

    job_args = [(dataset, curriculum, model_type, dataset_cache[dataset]) for dataset, curriculum, model_type in jobs]

    with Pool(10) as pool:
        results = list(tqdm(pool.imap(process_experiment, job_args), total=len(job_args)))

    df = pd.DataFrame(results)
    df.to_pickle(cache_path)

else:
    print("Reusing cached results")

df = pd.read_pickle(cache_path)


In [ ]:
# df = df.set_index(["dataset","curriculum"])

In [ ]:
import torch
import numpy as np
from scipy.stats import kendalltau

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import torch.nn.functional as F
def process_job(d):
    dataset_name, curriculum_name, model_type = d
    curriculum = get_curriculum(dataset_name,curriculum_name)
    return (d, curriculum)


data = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_job, d) for d in jobs]
    for future in tqdm(as_completed(futures), total=len(jobs), desc="Processing jobs"):
        data.append(future.result())


tensors = [item[1] for item in data]
file_names = [item[0] for item in data]



def compute_corr(i, j):
    a = tensors[i]
    b = tensors[j]
    stacked =[]
    for ta, tb in zip(a, b):
        min_len = min(ta.shape[0], tb.shape[0])
        stacked.append((ta[:min_len], tb[:min_len]))
    return (i, j, np.mean([kendalltau(s[0],s[1], 
        variant="b", # there will be ties
        alternative="two-sided", 
    )[0] for s in stacked]))

n = len(tensors)
pairs = [(i, j) for i in range(n) for j in range(i, n)]


results = []
with ThreadPoolExecutor(max_workers=128) as executor:
    results = list(tqdm(executor.map(lambda pair: compute_corr(pair[0], pair[1]), pairs), total=len(pairs), desc="Computing correlations"))


corr_matrix = np.zeros((n, n))
for i, j, corr in results:
    corr_matrix[i, j] = corr
    if i != j:
        corr_matrix[j, i] = corr


In [ ]:
import util

In [ ]:
multi_index = pd.MultiIndex.from_tuples(file_names, names=['dataset', 'curriculum',"model_type"])

corr_df = pd.DataFrame(corr_matrix, index=multi_index, columns=multi_index)


corr_df = corr_df.sort_index(level=1, ascending=True,key=lambda x: x.map(util.rename))
corr_df = corr_df.sort_index(level='curriculum', axis=1, ascending=True, key=lambda x: x.map(util.rename))



In [ ]:
import pandas as pd
from scipy.stats import ttest_ind


def p(group_by, melted,alternative):

    grouped = melted.groupby(group_by)['value_b']

    group_combinations = [x for x in grouped.groups.keys()]

    ttest_results = []
 
    for i in range(len(group_combinations)):
        for j in range(len(group_combinations)):
            group1 = grouped.get_group(group_combinations[i])
            group2 = grouped.get_group(group_combinations[j])
    
            if len(group_by) > 1 and (group_combinations[i][0:-1] != group_combinations[j][0:-1]):
                continue
            elif len(group_by) == 1 and group_combinations[i][0] == group_combinations[j][0]:
                    continue
            else:
                pass
  
            t_stat, p_value = ttest_ind(group1, group2,alternative=alternative,equal_var=False)
            ttest_results.append({
                'group_1': group_combinations[i],
                'group_2': group_combinations[j],
                'mean_1': np.mean(group1),
                'mean_2': np.mean(group2),
                't_stat': t_stat,
                'mean(1)-mean(2)': np.mean(group1) - np.mean(group2) ,
                'p_value': p_value,
                "significant": p_value < 0.05
            })

    ttest_results_df = pd.DataFrame(ttest_results)

    ttest_results_df['pair_key'] = ttest_results_df.apply(lambda row: tuple(sorted([row['group_1'], row['group_2']])), axis=1)
    ttest_results_df = ttest_results_df.drop_duplicates(subset='pair_key')
    ttest_results_df = ttest_results_df.drop(columns='pair_key')
    return ttest_results_df


In [ ]:
# what baseline curriculum are (increasing, decreasing) curricula most similar to? 
import util

melted = corr_df.melt(ignore_index=False).add_suffix("_b").reset_index()
melted= melted[melted.apply(lambda row: "influence" not in row["curriculum_b"],axis=1)]

melted= melted[melted.apply(lambda row: ("bins" in row["curriculum"]) or ("_cp_dirac" in row["curriculum"]),axis=1)]
melted["group"] = melted.apply(lambda row: "incr" if "incr" in row["curriculum"] else "decr" ,axis=1)

group_by = ["curriculum_b", "group"]  
f = p(group_by, melted,alternative="greater")
a = f[f["group_1"] != f["group_2"]]
numeric_cols = a.select_dtypes(include=[np.number]).columns
a.style.format({col: "{:.3f}" for col in numeric_cols})

In [ ]:
# does filtering with lognorm filter increase similarity to baselines?

melted = corr_df.melt(ignore_index=False).add_suffix("_b").reset_index()
melted= melted[melted.apply(lambda row: "influence" not in row["curriculum_b"],axis=1)]
melted= melted[melted.apply(lambda row: "influence" in row["curriculum"],axis=1)]

melted= melted[melted.apply(lambda row: ("decr_bins_lognorm" in row["curriculum"]) or ("decr_bins_dirac" in row["curriculum"]),axis=1)]


melted["group"] = melted.apply(lambda row: "lognorm" if "lognorm" in row["curriculum"] else "raw" ,axis=1)

group_by = ["curriculum_b", "group"]  
f = p(group_by, melted,alternative="greater")
a = f[f["group_1"] != f["group_2"]]
numeric_cols = a.select_dtypes(include=[np.number]).columns
a.style.format({col: "{:.3f}" for col in numeric_cols})

In [ ]:
# is similarity to baselines for a curriculum greater for RoBERTa than Llama?

melted = corr_df.melt(ignore_index=False).add_suffix("_b").reset_index()
melted= melted[melted.apply(lambda row: "influence" not in row["curriculum_b"],axis=1)]
melted= melted[melted.apply(lambda row: "influence" in row["curriculum"],axis=1)]

melted= melted[melted.apply(lambda row: ("_cp_dirac" in row["curriculum"]),axis=1)]


melted["group"] = melted.apply(lambda row: "RoBERTa" if "roberta" in row["curriculum"] else "Llama" ,axis=1)
for a in (melted.groupby("group")["curriculum"].unique()):
    print(a)

group_by = ["curriculum_b", "group"]  
f = p(group_by, melted,alternative="two-sided")
a = f[f["group_1"] != f["group_2"]]
numeric_cols = a.select_dtypes(include=[np.number]).columns

a.style.format({col: "{:.3f}" for col in numeric_cols})


In [ ]:
# how do curricula for roberta correlate with those for llama?
corr_df

In [ ]:
melted

In [ ]:

plt.figure(figsize=(12, 12))
sns.heatmap(corr_df, annot=False, cbar=True)

plt.xticks(rotation=45, ha='right')  
plt.yticks(rotation=0)  
plt.tight_layout()  
plt.show()


In [ ]:
import util

In [ ]:
# how similar are the curricula across model types?
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import numpy as np
import pandas as pd

all_matrices = []

unique_datasets = corr_df.index.get_level_values('dataset').unique()
unique_model_types = corr_df.index.get_level_values('model_type').unique()

for dataset in unique_datasets:

    dataset_corr = corr_df.loc[
        (dataset, ["roberta" + c for c in config.influence_curricula], "roberta"),
        (dataset, ["llama" + c for c in config.influence_curricula], "llama")
    ]



    midx = pd.MultiIndex.from_tuples(list(zip(dataset_corr.index,dataset_corr.columns)))
    dataset_corr = pd.DataFrame(data=np.diag(dataset_corr), index=midx).T


    # index_labels = [util.rename(i) for i in dataset_corr.index.get_level_values(1).unique()]
    # column_labels = [util.rename(i) for i in dataset_corr.columns.get_level_values(1).unique()]
    fig = plt.figure(figsize=(10, 8))
    gs = gridspec.GridSpec(2, 1, height_ratios=[5, 1], hspace=0)
    ax = fig.add_subplot(gs[0])
    cbar_ax = fig.add_subplot(gs[1])

    sns.heatmap(
        dataset_corr,
        ax=ax,
        cbar_ax=cbar_ax,
        annot=True,
        cmap='RdGy',
        fmt='.2f',
        cbar_kws={'orientation': 'horizontal'},
        linewidths=0.5,
        annot_kws={'size': 12},
        vmin=-0.5,
        vmax=0.5
    )

   # ax.set_yticklabels(index_labels, rotation=0, fontsize=14)
    # ax.set_xticklabels(column_labels, rotation=45, fontsize=14)
    print(util.rename_dataset(dataset))
    plt.xlabel("", fontsize=14)
    plt.ylabel("$\\tau$", fontsize=14)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel("")
    ax.set_ylabel("")

    save_path = os.path.join("./autogenerated_figures", dataset ,"kendalltau_cross_model_type.pdf")
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    plt.savefig(save_path, dpi=600, bbox_inches='tight')
    plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

unique_datasets = corr_df.index.get_level_values('dataset').unique()
unique_model_types = corr_df.index.get_level_values('model_type').unique()

for dataset in unique_datasets:
    for model_type in unique_model_types:
        dataset_corr = corr_df.loc[
            (dataset, config.baseline_curricula, model_type),
            (dataset, [model_type + c for c in config.influence_curricula], model_type)
        ]

        fig = plt.figure(figsize=(10, 8))
        gs = gridspec.GridSpec(2, 1, height_ratios=[20, 1], hspace=-0.10)
        ax = fig.add_subplot(gs[0])
        cbar_ax = fig.add_subplot(gs[1])

        sns.heatmap(
            dataset_corr,
            ax=ax,
            cbar_ax=cbar_ax,
            annot=True,
            cmap='RdGy',
            fmt='.2f',
            cbar_kws={'orientation': 'horizontal'},
            linewidths=0.5,
            annot_kws={'size': 12},
            vmin=-1,
            vmax=1
        )

        ax.set_yticklabels(
            [util.rename(i) for i in dataset_corr.index.get_level_values(1).unique()],
            rotation=0,
            fontsize=14
        )
        ax.set_xticklabels(
            [util.rename(i) for i in dataset_corr.columns.get_level_values(1).unique()],
            rotation=45,
            fontsize=14
        )


        plt.xlabel("", fontsize=14)
        plt.ylabel("", fontsize=14)
        ax.set_aspect('equal', adjustable='box')
        ax.set_xlabel("")
        ax.set_ylabel("")
        save_path = os.path.join("./autogenerated_figures", "kendalltau", dataset, model_type + ".pdf")
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        

        plt.savefig(save_path, dpi=600, bbox_inches='tight')
        plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import numpy as np
import pandas as pd

all_matrices = []

unique_datasets = corr_df.index.get_level_values('dataset').unique()
unique_model_types = corr_df.index.get_level_values('model_type').unique()

for dataset in unique_datasets:
    for model_type in unique_model_types:
        try:
            dataset_corr = corr_df.loc[
                (dataset, config.baseline_curricula, model_type),
                (dataset, [model_type + c for c in config.influence_curricula], model_type)
            ]
            all_matrices.append(dataset_corr.values)
        except KeyError:
            continue

mean_matrix = np.mean(all_matrices, axis=0)

example = all_matrices[0]
index_labels = [util.rename(i) for i in dataset_corr.index.get_level_values(1).unique()]
column_labels = [util.rename(i) for i in dataset_corr.columns.get_level_values(1).unique()]
fig = plt.figure(figsize=(10, 8))
gs = gridspec.GridSpec(2, 1, height_ratios=[20, 1], hspace=-0.10)
ax = fig.add_subplot(gs[0])
cbar_ax = fig.add_subplot(gs[1])

sns.heatmap(
    mean_matrix,
    ax=ax,
    cbar_ax=cbar_ax,
    annot=True,
    cmap='RdGy',
    fmt='.2f',
    cbar_kws={'orientation': 'horizontal'},
    linewidths=0.5,
    annot_kws={'size': 12},
    vmin=-0.2,
    vmax=0.2
)

ax.set_yticklabels(index_labels, rotation=0, fontsize=14)
ax.set_xticklabels(column_labels, rotation=45, fontsize=14)

plt.xlabel("", fontsize=14)
plt.ylabel("$\\tau$", fontsize=14)
ax.set_aspect('equal', adjustable='box')
ax.set_xlabel("")
ax.set_ylabel("")

save_path = os.path.join("./autogenerated_figures", "kendalltau_mean.pdf")
os.makedirs(os.path.dirname(save_path), exist_ok=True)

plt.savefig(save_path, dpi=600, bbox_inches='tight')
plt.show()
